In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Preview data

In [ ]:
from weaviate.classes.query import Filter

wiki = client.collections.get("WikipediaPQ")

response = wiki.query.fetch_objects(
    filters=Filter.by_property("text").like("musical"),
    limit=5
)

for item in response.objects:
    print(item.properties["wiki_id"])
    print(item.properties["title"])
    print(item.properties["text"], '\n')

## Vector search

In [ ]:
wiki = client.collections.get("WikipediaPQ")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print(item.properties)

> Have fun! Add your own queries.

In [ ]:
client.close()